In [ ]:
#FileB- Classification using Machine Learning

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df=pd.read_csv('FILEB.csv')
df.head(2)

,ProjectNumber,Lead,RecipientName,RecipientType,Indigenous,Town_City_Community,Province_Territory,PostalCode,ContactName,ContactPhone,ContactEmail,Funding_Status,Requested,Approved_Funding,Investment_Type,Investment_Description,Investment_Output,Describe_Remaining_Need,Additonal_Funding_Needed,PageIndex
0,521-27,Breakfast Club of Canada,NaN,NaN,No,Toronto,ON,NaN,NaN,NaN,NaN,Approved,NaN,1082319.0,Financial Assistance,NaN,NaN,NaN,NaN,46&47
1,521-1335,Breakfast Club of Canada,NaN,NaN,Yes,Vancouver,BC,NaN,NaN,NaN,NaN,Approved,NaN,251702.0,Financial Assistance,NaN,NaN,NaN,NaN,46&47


In [ ]:
print(df.shape)
print("\n")
print(df.columns)

(3513, 20)


Index(['ProjectNumber', 'Lead', 'RecipientName', 'RecipientType', 'Indigenous',
       'Town_City_Community', 'Province_Territory', 'PostalCode',
       'ContactName', 'ContactPhone', 'ContactEmail', 'Funding_Status',
       'Requested', 'Approved_Funding', 'Investment_Type',
       'Investment_Description', 'Investment_Output',
       'Describe_Remaining_Need', 'Additonal_Funding_Needed', 'PageIndex'],
      dtype='object')


In [ ]:
#remove columns not required for analysis
dfb = df.drop(columns=['ProjectNumber','RecipientName', 'RecipientType', 'Indigenous', 'Province_Territory', 'PostalCode',
       'ContactName', 'ContactPhone', 'ContactEmail','Requested', 'Investment_Description',
       'Investment_Output', 'Describe_Remaining_Need',
       'Additonal_Funding_Needed', 'PageIndex'])
dfb.reset_index(drop=True,inplace=True)

In [ ]:
dfb.Investment_Type.value_counts()

Food & Meals                     1148
Financial Assistance             1032
Food+                             855
Transportation/Storage            138
Wages                             111
Personal Protective Equipment     101
Capital Investment                 62
Food Distribution                  39
Name: Investment_Type, dtype: int64

In [ ]:
dfb.Lead.value_counts()

Breakfast Club of Canada         942
Second Harvest                   930
Food Banks Canada                838
The Salvation Army               460
Community Food Centres Canada    314
La Tablee des Chefs               21
Name: Lead, dtype: int64

In [ ]:
#ML operations

In [ ]:
#creating a copy of dataframe
data=dfb
data.Funding_Status.unique()

array(['Approved', 'Approved- other', 'rejected', 'reiected',
       'D - Funded by another AAFC ore', 'D - Duplicate Apo',
       'D - Did Not Qualifv', 'D - Incomplete Apo',
       'D - Funded by another AAFC org', 'Did Not Qualifv',
       'Affiliated with another AAFC funded org', 'Withdrawn',
       'Duplicate Apo', 'Did Not Qualify', 'Duplicate App',
       'D - Did Not Qualify', 'Duplicate Apn',
       'D - Affiliated with another AAFC funded org', 'Incomplete Apo',
       'D - Did Not Quality', 'Funded by another AAFC org',
       'Did Not Quality', 'Incomplete App', 'D - Incomplete App',
       'Duplicate Lnn', 'D - Duplicate App', 'D - Duplicate Apn',
       'Funded by another org AAFC _', 'D - Funded by another org AAFC _',
       'D - Affiliated with another AAFC tunded org'], dtype=object)

In [ ]:
#feature-transformation

# Define the transformation function
def transform_funding_status(x):
    if x in ['Approved', 'Approved-Other']:
        return 'Approved'
    else:
        return 'NotApproved'

# Apply the transformation to the 'Funding_Status' column
data['Funding_Status'] = data['Funding_Status'].apply(lambda x: transform_funding_status(x))

# Display the updated DataFrame values for Funding_Status
data.Funding_Status.value_counts()


Approved       3255
NotApproved     258
Name: Funding_Status, dtype: int64

In [ ]:
#downsampling


# Count the number of 'Approved' and 'NotApproved' rows in the 'Funding_Status' column
counts = data['Funding_Status'].value_counts()

# Determine the size of the downsampled 'Approved' subset
n_approved = counts['Approved']
n_not_approved = counts['NotApproved']
downsampled_size = n_not_approved

# Randomly select a subset of the 'Approved' rows to keep
approved_subset = data[data['Funding_Status'] == 'Approved'].sample(n=downsampled_size)

# Select all the 'Not Approved' rows
not_approved_subset = data[data['Funding_Status'] == 'NotApproved']

# Concatenate the 'Approved' and 'Not Approved' subsets into a single DataFrame
downsampled_data = pd.concat([approved_subset, not_approved_subset])

# Shuffle the rows of the downsampled DataFrame
downsampled_data = downsampled_data.sample(frac=1, random_state=42)
downsampled_data.shape

(516, 5)

In [ ]:
downsampled_data.Funding_Status.value_counts()

NotApproved    258
Approved       258
Name: Funding_Status, dtype: int64

In [ ]:
#Classification using Logistic Regression

In [ ]:
import pandas as pd

X = downsampled_data[['Lead', 'Town_City_Community', 'Approved_Funding', 'Investment_Type']]
y = downsampled_data['Funding_Status']


In [ ]:
#preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Encode categorical variables
le = LabelEncoder()
X['Lead'] = le.fit_transform(X['Lead'])
X['Town_City_Community'] = le.fit_transform(X['Town_City_Community'])
X['Investment_Type'] = le.fit_transform(X['Investment_Type'])

# Handle missing values
imputer = SimpleImputer()
X = imputer.fit_transform(X)

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

<ipython-input-27-760baaf6ef97>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Lead'] = le.fit_transform(X['Lead'])
<ipython-input-27-760baaf6ef97>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Town_City_Community'] = le.fit_transform(X['Town_City_Community'])
<ipython-input-27-760baaf6ef97>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train the model
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

In [ ]:
# Evaluate the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix,classification_report

print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred,pos_label='Approved',average='micro'))

Accuracy: 0.8372093023255814
F1 Score: 0.8372093023255814


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'Approved') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Approved       0.98      0.72      0.83        71
 NotApproved       0.74      0.98      0.84        58

    accuracy                           0.84       129
   macro avg       0.86      0.85      0.84       129
weighted avg       0.87      0.84      0.84       129

